In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os, json,time



# -----------------------
# MOCK AGENTS (Replace with actual imports if available)
# -----------------------
def run_summary_agent(workflow_name, session):
    """Mocks the output of the actual summary agent."""
    return f"Detailed summary for '{workflow_name}' generated successfully."

def run_sttm_agent(workflow_name, session):
    """Mocks the output of the actual STTM agent."""
    
    return f"Source-to-Target Mapping (STTM) report for '{workflow_name}' generated successfully."

def run_object_lineage_agent(workflow_name, session):
    """Mocks the output of the actual Object Lineage agent."""
    return f"Object Lineage for '{workflow_name}' generated successfully."


# -----------------------
# 1️⃣ Setup
# -----------------------
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)


# -----------------------
# 2️⃣ Tools
# -----------------------

@tool("summary_tool", return_direct=False)
def summary_tool(workflow_name: str, session: dict) -> str:
    """ 
    Generates a detailed summary for a given Informatica workflow session.

    Parameters
    ----------
    workflow_name : str
        Name of the Informatica workflow being analyzed.
    session : dict
        Complete session path in dictionary form (including mapplet paths).
    
    Returns
    ----------
        summary generated message
    """
    print(f"Summary agent started for {workflow_name}........")
    time.sleep(0.5) # Simulate work
    response = run_summary_agent(workflow_name, session)
    print("Summary agent completed........")
    return response


@tool("sttm_tool", return_direct=False)
def sttm_tool(workflow_name: str, session: dict) -> str:
    """ 
    Generates a Source-to-Target Mapping (STTM) for the given Informatica session.

    Parameters
    ----------
    workflow_name : str
        Workflow name associated with the session.
    session : dict
        Complete session dictionary containing mapping information.
    Returns
    ----------
        sttm generated message
    """
    print(f"STTM agent started for {workflow_name}........")
    time.sleep(0.5) # Simulate work
    response = run_sttm_agent(workflow_name,session)
    print("STTM agent completed........")
    return response


@tool("object_lineage_tool", return_direct=False)
def object_lineage_tool(workflow_name: str, session: dict) -> str:
    """ 
    Generates a Object Lineage for the given Informatica session.

    Parameters
    ----------
    workflow_name : str
        Workflow name associated with the session.
    session : dict
        Complete session dictionary containing mapping information.
    Returns
    ----------
        object lineage generated message
    """
    print(f"Object Lineage agent started for {workflow_name}........")
    time.sleep(0.5) # Simulate work
    response = run_object_lineage_agent(workflow_name,session)
    print("Object Lineage agent completed........")
    return response
    


# -----------------------
# 3️⃣ Analyzer Orchestrator Agent
# -----------------------
def run_analyzer_agent(workflow_name: str, session: dict, user_request: str)-> str:
    """
    Analyzer Orchestrator Agent

    Dynamically determines which tools to execute (summary_tool, sttm_tool, object_lineage_tool)
    based on user intent and ensures termination after completion.
    """
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are an Analyzer Orchestrator agent. Your primary goal is to **execute all requested tools** and then **STOP**. "
            "You have access to tools and must use them to solve the task. "
            "Once you have received observations confirming all required analyses are complete, you MUST use the Final Answer format to stop the execution. "
            "Example of the final output format: **Final Answer: Analysis completed successfully.**"
        ),
        (
            "human",
            """
            You are a **Analyzer Orchestrator Agent** responsible for determining and executing 
            the correct analysis tool(s) based on the user’s request.
            
            **CRITICAL RULE:** After executing all required tools and receiving the observations, you MUST stop and return the final answer. **DO NOT** attempt to re-run any tools.
            
            ### Available Tools
            - `summary_tool(workflow_name, session)` → Generates a textual summary of the session logic.
            - `sttm_tool(workflow_name, session)` → Generates a Source-to-Target Mapping (STTM) report.
            - `object_lineage_tool(workflow_name, session)` → Generates a Object Lineage of the session.
            
            ### Decision Rules
            1. If the user’s request mentions "summary", "overview", or "describe" → Call **summary_tool** only.
            2. If the user’s request mentions "sttm", or "source to target mapping" → Call **sttm_tool** only.
            3. If the user’s request mentions "object lineage", "mapping lineage", or "lineage" → Call **object_lineage_tool** only.
            4. If the request is general (e.g., "analyze", "process", or "analyze session") → Execute **all the three tools sequentially**.
            
            ---
            
            Always pass parameters as:
            - `workflow_name`: {workflow_name}
            - `session`: {session} (You MUST pass this as a JSON object, not a string)
            
            ### Inputs
            User Request: {user_request}

            {agent_scratchpad}
            """
        )]
    )

    # ALL three tools must be registered here
    all_tools = [summary_tool, sttm_tool, object_lineage_tool]
    
    orchestrator = create_tool_calling_agent(
        llm=llm,
        tools=all_tools,
        prompt=prompt
    )

    executor = AgentExecutor(
        agent=orchestrator,
        tools=all_tools,
        handle_parsing_errors=True,
        # Max 3 tools + Final Answer step -> 6 is a safe limit.
        max_iterations=6,  
#         max_execution_time=20.0, # Added time fail-safe
        verbose=True # Changed to True to see the steps
    )
    
    result = executor.invoke({
            "workflow_name": workflow_name,
            "session": session,
            "user_request": user_request
        }) 
    return result["output"]


if __name__ == "__main__":
    workflow_name = 'wf_AGENT_LOAD'
    session = {
                "session name": "s_m_T",
                "mapping name": "m_T",
                "session path": "reconstructed_session_xmls\\s_m_T.xml",
                "canonical model path": "canonical_models\\s_m_T.json",
                "mapplets": [
                    {
                        "mapplet name": "mplt",
                        "mapplet canonical model path": "canonical_models\\mplt.json"
                    }
                ]
            }
    
    # --------------------------------------------------------------------------------
    # NOTE ON USER'S ORIGINAL QUERY: "analyze and convert informatica mapping into dbt"
    # This query includes a conversion tool ("dbt") that is not in this agent's scope.
    # The agent will likely only perform the "analyze" part (calling all 3 analysis tools)
    # and then mention it cannot fulfill the "dbt" part. This is correct behavior for this agent.
    # --------------------------------------------------------------------------------

    user_request = "generate summary and sttm document for the given informatica session"
    
    print("\n" + "="*70)
    print("RUNNING TWO-TOOL REQUEST (Summary, STTM)")
    print("="*70)
    res_three = run_analyzer_agent(workflow_name, session, user_request)
    print("\n--- FINAL AGENT OUTPUT (Three-tool) ---")
    print(res_three)
    print("--------------------------------------\n")
    
    # Example using one tool
    user_request_single = "Generate only the object lineage report for this workflow."
    print("\n" + "="*70)
    print("RUNNING SINGLE-TOOL REQUEST (Lineage)")
    print("="*70)
    res_single = run_analyzer_agent(workflow_name, session, user_request_single)
    print("\n--- FINAL AGENT OUTPUT (Single-tool) ---")
    print(res_single)
    print("--------------------------------------\n")